# <center>Tabular Playground Series - June/2021<center>
## <center>Keras Neural Network with Embedding Layer<center>
---

Neural Network inspired by [@pourchot's](https://www.kaggle.com/pourchot) notebook [Simple Keras embedding in 10 folds](https://www.kaggle.com/pourchot/simple-keras-embedding-in-10-folds). If you find this notebook useful, please consider upvoting both this and [@pourchot's](https://www.kaggle.com/pourchot)

My other notebooks in this competition:
- [Tabular Playground Series - June/2021: Starter - EDA + Base LightGBM](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-starter-eda-base-lgbm)
- [Tabular Playground Series - June/2021: Simple Neural Network with Keras](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-simple-nn-with-keras)
- [Tabular Playground Series - June/2021: Wide and Deep Neural Network with Keras](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-wide-and-deep-nn-w-keras)
- [Tabular Playground Series - June/2021: LightAutoML with KNN Features](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-lightautoml-w-knn-feats)
- [Tabular Playground Series - June/2021: Keras Neural Network with Skip Connections](https://www.kaggle.com/jonaspalucibarbosa/tps06-21-keras-nn-with-skip-connections)


## Importing Libraries and Datasets

In [ ]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns
%matplotlib inline

import random
import os

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks
from keras import backend as K

In [ ]:
#Reproducible results
from numpy.random import seed
seed(42)
from tensorflow.random import set_seed
set_seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [ ]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv', index_col = 'id')
Y_train = df_train['target'].copy()
X_train = df_train.copy().drop('target', axis = 1)

X_test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv', index_col = 'id')

In [ ]:
class_map = {'Class_1': 0,
            'Class_2': 1,
            'Class_3': 2,
            'Class_4': 3,
            'Class_5': 4,
            'Class_6': 5,
            'Class_7': 6,
            'Class_8': 7,
            'Class_9': 8}
Y_train = Y_train.map(class_map).astype('int')
Y_train

In [ ]:
#Converting target series to matrix for multiclass classification on Keras

Y_train = to_categorical(Y_train)
Y_train

## Creating and Evaluating the NN

In [ ]:
cce = keras.losses.CategoricalCrossentropy()
#Custom metric from Pourchot added 24/06
def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

early_stopping = callbacks.EarlyStopping(
    monitor='val_custom_metric',
    patience=12,
    min_delta=0.0000001,
    restore_best_weights=True,
)

plateau = callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric',
    factor = 0.5,                                     
    patience = 2,                                   
    min_delt = 0.0000001,                                
    cooldown = 0,                               
    verbose = 1
) 

In [ ]:
def get_model():
    model = keras.Sequential([
        layers.Input(shape = (75,)),
        layers.Embedding(360, 8, ),
        layers.Conv1D(16, kernel_size=1, activation='relu'), #added 22/06
        layers.Flatten(),
        layers.Dropout(0.3), #added 20/06
        #layers.BatchNormalization(input_shape = [75]),
        layers.Dense(units = 128, activation = 'relu', kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(units = 64, activation = 'relu', kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(units = 32, activation = 'relu', kernel_initializer='he_normal'),        
        layers.BatchNormalization(),
        layers.Dropout(0.2),    
        layers.Dense(9, activation = 'softmax', kernel_initializer='lecun_normal'), #init add 27/06 based on BIZEN
    ])
    
    return model

In [ ]:
keras.backend.clear_session()

model = get_model()
model.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=0.0002), metrics=custom_metric)

model.summary()

In [ ]:
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42
                                                    , stratify = Y_train)

In [ ]:
history = model.fit(X_train_split, Y_train_split,
          batch_size = 128, epochs = 100,
          validation_data=(X_val_split, Y_val_split),
          callbacks=[early_stopping, plateau]);

In [ ]:
score = model.evaluate(X_val_split, Y_val_split, verbose = 0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}%'.format(score[1] * 100))

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x = history.epoch, y = history.history['loss'])
sns.lineplot(x = history.epoch, y = history.history['val_loss'])
ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.legend(['train', 'test'], loc='best')
plt.show()

## Making Predictions

In [ ]:
Y_train = df_train['target'].copy()
Y_train = Y_train.map(class_map).astype('int')
Y_train

In [ ]:
def prediction (X_train, Y_train, X_test):
    
    keras.backend.clear_session()

    kfold = StratifiedKFold(n_splits = 25)

    y_pred = np.zeros((100000,9))
    train_oof = np.zeros((200000,9))
    
    for idx in kfold.split(X=X_train, y=Y_train):
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X_train.iloc[train_idx]
        ytrain = Y_train.iloc[train_idx]
        xval = X_train.iloc[val_idx]
        yval = Y_train.iloc[val_idx]
        
        ytrain = to_categorical(ytrain)
        yval = to_categorical(yval)
        
        # fit model for current fold
        model = get_model()
        model.compile(loss='categorical_crossentropy'
                      , optimizer = keras.optimizers.Adam(learning_rate=0.0002), metrics=custom_metric)
        
        model.fit(xtrain, ytrain,
        batch_size = 128, epochs = 100,
        validation_data=(xval, yval),
        callbacks=[early_stopping, plateau]);

        #create predictions
        y_pred += model.predict(X_test)/kfold.n_splits
        print(y_pred)
               
        val_pred = model.predict(xval)
        # getting out-of-fold predictions on training set
        train_oof[val_idx] = val_pred
        
        # calculate and append logloss
        fold_logloss = metrics.log_loss(yval,val_pred)
        print("Logloss: {0:0.5f}". format(fold_logloss))
  
    return y_pred, train_oof

In [ ]:
nn_pred, train_oof = prediction (X_train, Y_train, X_test)

In [ ]:
print("Logloss: {0:0.6f}".format(metrics.log_loss(Y_train,train_oof)))

In [ ]:
train_oof = pd.DataFrame(train_oof, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
train_oof

In [ ]:
pred_test = pd.DataFrame(nn_pred, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
pred_test

In [ ]:
train_oof.to_csv('nn_train_oof.csv', index=False)
train_oof

In [ ]:
output = pred_test
output['id'] = X_test.index
output.to_csv('submission.csv', index=False)

output